In [15]:
!pip install --quiet "python-telegram-bot==20.7" "google-cloud-dialogflow"


ERROR:asyncio:Task was destroyed but it is pending!
task: <Task pending name='Application:8227320998:update_fetcher' coro=<Application._update_fetcher() running at /usr/local/lib/python3.12/dist-packages/telegram/ext/_application.py:1192> wait_for=<Future pending cb=[Task.__wakeup()]>>


In [16]:
from google.colab import files
import os

print("Upload your Dialogflow service account JSON file:")
uploaded = files.upload()

service_account_file = list(uploaded.keys())[0]
print("Using credentials file:", service_account_file)

# Let Dialogflow SDK pick up the credentials
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = service_account_file

# 🔧 CONFIG – EDIT THESE
DIALOGFLOW_PROJECT_ID = "customer-support-bot-vvm9"    # your Dialogflow project ID
TELEGRAM_BOT_TOKEN = "8227320998:AAEg58uxAoqmM756M0tMF5-ZjySV6pa7NBE"  # from BotFather

LANGUAGE_CODE = "en"          # or "en-US" if that’s what your agent uses
SESSION_ID_PREFIX = "tg-"     # used to maintain context per Telegram chat


Upload your Dialogflow service account JSON file:


Saving customer-support-bot-vvm9-8537d4a66e32.json to customer-support-bot-vvm9-8537d4a66e32 (7).json
Using credentials file: customer-support-bot-vvm9-8537d4a66e32 (7).json


In [17]:
from google.cloud import dialogflow_v2 as dialogflow

def detect_intent_text(project_id: str, session_id: str, text: str, language_code: str = "en") -> str:
    """
    Sends user text to Dialogflow and returns the agent's reply text.
    This uses whatever intents, responses, and fallback you configured.
    """
    session_client = dialogflow.SessionsClient()
    session = session_client.session_path(project_id, session_id)

    text_input = dialogflow.TextInput(text=text, language_code=language_code)
    query_input = dialogflow.QueryInput(text=text_input)

    response = session_client.detect_intent(
        request={
            "session": session,
            "query_input": query_input
        }
    )

    fulfillment_text = response.query_result.fulfillment_text
    if not fulfillment_text:
        fulfillment_text = "Sorry, I didn't understand that."

    return fulfillment_text


In [ ]:
import asyncio
from telegram import Update
from telegram.ext import (
    ApplicationBuilder,
    CommandHandler,
    MessageHandler,
    ContextTypes,
    filters,
)

# /start -> trigger a greeting via Dialogflow
async def start(update: Update, context: ContextTypes.DEFAULT_TYPE):
    chat_id = update.effective_chat.id
    session_id = SESSION_ID_PREFIX + str(chat_id)

    df_reply = detect_intent_text(
        project_id=DIALOGFLOW_PROJECT_ID,
        session_id=session_id,
        text="hi",  # this will hit your welcome/hi intent or training phrases
        language_code=LANGUAGE_CODE,
    )

    await context.bot.send_message(chat_id=chat_id, text=df_reply)

# /help -> optional static help
async def help_command(update: Update, context: ContextTypes.DEFAULT_TYPE):
    chat_id = update.effective_chat.id
    help_text = "You can ask any customer support question that the Dialogflow bot is trained on."
    await context.bot.send_message(chat_id=chat_id, text=help_text)

# Any normal text -> Dialogflow (intents + fallback)
async def handle_message(update: Update, context: ContextTypes.DEFAULT_TYPE):
    chat_id = update.effective_chat.id
    user_text = update.message.text
    session_id = SESSION_ID_PREFIX + str(chat_id)

    try:
        df_reply = detect_intent_text(
            project_id=DIALOGFLOW_PROJECT_ID,
            session_id=session_id,
            text=user_text,
            language_code=LANGUAGE_CODE,
        )
    except Exception as e:
        print("Dialogflow error:", e)
        df_reply = "Sorry, something went wrong while talking to the support system."

    await context.bot.send_message(chat_id=chat_id, text=df_reply)

async def main():
    application = (
        ApplicationBuilder()
        .token(TELEGRAM_BOT_TOKEN)
        .build()
    )

    application.add_handler(CommandHandler("start", start))
    application.add_handler(CommandHandler("help", help_command))
    application.add_handler(MessageHandler(filters.TEXT & ~filters.COMMAND, handle_message))

    print("Bot started. Open Telegram and talk to your bot.")
    await application.run_polling()

await main()


Bot started. Open Telegram and talk to your bot.


ERROR:telegram.ext.Updater:Error while getting Updates: httpx.ReadError: 
ERROR:telegram.ext.Application:No error handlers are registered, logging exception.
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/httpx/_transports/default.py", line 101, in map_httpcore_exceptions
    }
      
  File "/usr/local/lib/python3.12/dist-packages/httpx/_transports/default.py", line 394, in handle_async_request
  File "/usr/local/lib/python3.12/dist-packages/httpcore/_async/connection_pool.py", line 256, in handle_async_request
    raise exc from None
  File "/usr/local/lib/python3.12/dist-packages/httpcore/_async/connection_pool.py", line 236, in handle_async_request
    response = await connection.handle_async_request(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/httpcore/_async/connection.py", line 103, in handle_async_request
    return await self._connection.handle_async_request(request)
           ^^^^^^